In [1]:
import time
print(time.ctime(time.time()))

Tue Dec  9 22:07:32 2025


# Advent of Code Day 9

Puzzle text available at:
https://adventofcode.com/2025/day/9

In [2]:
import os
import numpy as np
from itertools import combinations, pairwise

In [3]:
# Day of calendar
day = 9
# Input filename
puzzle_test = 'input_day%02d_test.txt' %(day)
puzzle_input = 'input_day%02d.txt' %(day)

In [4]:
red_tiles = []
with open(puzzle_input, 'r') as f:
    lines = f.readlines() 
for line in lines:
    red_tiles.append([int(x) for x in line.strip().split(',')])

## Part 1

Find the closest junction boxes and connect them

In [5]:
def get_area(point1,point2):
    dx = abs(point2[0] - point1[0]) + 1
    dy = abs(point2[1] - point1[1]) + 1

    area = dx * dy
    
    return area

In [6]:
# This part is quite simple so lets just loop through all the possible rectangles and check its area
corners = list(combinations(red_tiles, r=2))

area_max = 0
for point1, point2 in corners:
    area_new = get_area(point1, point2)
    area_max = max(area_max,area_new)

In [7]:
print("What is the largest area of any rectangle you can make?")
print(int(area_max))

What is the largest area of any rectangle you can make?
4763040296


## Part 2

Now we want the largest area within the limits of the red tiles. There are two possible ways:  
- 1) Determine if the rectangle interesects any "green" union between red tiles. If don't the rectangle is inside and hence valid 
- 2) Using shapely.Polygon

### Solution 1

In [8]:
corners = [([min(p1,q1), min(p2,q2)], [max(p1,q1), max(p2,q2)]) for (p1,p2),(q1,q2) in combinations(red_tiles, r=2)]
consecutive_pairs = [([min(a1,b1), min(a2,b2)], [max(a1,b1), max(a2,b2)]) for (a1,a2),(b1,b2) in pairwise(red_tiles + [red_tiles[0]])]

In [9]:
# Check if line segment (x1,y1)-(x2,y2) intersects segment (a,b)-(c,d)
def segment_interesect(p,q,a,b):
    """
    (p,q): Recctangle coordinates
    (a,b): Green tile edge coordinates
    """
    # return p[0] < b[0] and p[1] > a[0] and q[0] < b[1] and q[1] > a[1]
    return a[0] < q[0] and a[1] < q[1] and b[0] > p[0] and b[1] > p[1]

To increase the performance of the code we can sort in decreasing order the "area" formed by corners and green tiles,
technically green tile lines do not have area but we can consider them rectangles of size 1

Why in decrease order?
 - Corners: We are interested in the largest area possible so makes sense to start from the beginning
 - Gren tile lines: The longer the line, higher the chances of the rectangle crossing it,
if so we pass to the next rectangle sooner in the loop

In [10]:
# Define sorting by area function
def sort_by_area(pairs):
    """Sort pairs by decreasing area"""
    areas = [get_area(p1, p2) for p1, p2 in pairs]
    idx = np.argsort(areas)[::-1]
    return [pairs[i] for i in idx]

# Sort both sets of points
sorted_corners = sort_by_area(corners)
sorted_green_lines = sort_by_area(consecutive_pairs)

print(f'Sanity check: {get_area(sorted_corners[0][0],sorted_corners[0][1]) == area_max}')

Sanity check: True


In [11]:
# Loop through every green tile edge for every rectangle
for p,q in sorted_corners:
    for a,b in sorted_green_lines:
        # If the rectangle crosses the edge go to next one
        if segment_interesect(p, q, a, b):
            break
    else:
        result_part2 = get_area(p,q)
        break

In [12]:
print("What is the largest area of any rectangle you can make using only red and green tiles?")
print(int(result_part2))

What is the largest area of any rectangle you can make using only red and green tiles?
1396494456


### Solution 2 - Way Slower!!!

In [20]:
from shapely.geometry import Polygon, box, LineString

In [21]:
# Create the polygon from red tiles
red_polygon = Polygon(red_tiles)

# Sort corners by area (largest first)
corners = [([min(p1,q1), min(p2,q2)], [max(p1,q1), max(p2,q2)]) for (p1,p2),(q1,q2) in combinations(red_tiles, r=2)]
# Define sorting by area function
def sort_by_area(pairs):
    """Sort pairs by decreasing area"""
    areas = [get_area(p1, p2) for p1, p2 in pairs]
    idx = np.argsort(areas)[::-1]
    return [pairs[i] for i in idx]
sorted_corners = sort_by_area(corners)

# Find largest rectangle that doesn't intersect the polygon
for p, q in sorted_corners:
    rect = box(p[0], p[1], q[0], q[1])
    
    # Check each polygon edge as a line segment
    intersects_edge = False
    for i in range(len(red_tiles)):
        edge = LineString([red_tiles[i], red_tiles[(i+1) % len(red_tiles)]])
        
        # Check if rectangle interior intersects the edge
        # (excludes touching at corners/edges)
        if rect.intersects(edge) and not rect.touches(edge):
            intersects_edge = True
            break
    
    if not intersects_edge:
        result_part2_b = get_area(p, q)
        break

print(f'Part 2: {result_part2_b}')

Part 2: 1396494456
